In [ ]:
from os import makedirs
from os.path import join, exists
from itertools import product
import numpy as np
import pandas as pd

from scipy.stats import ttest_ind
import statsmodels.api as sm

from brainpipe.system import study
from mne.stats import fdr_correction, bonferroni_correction
from utils import rename_elecs, odors_su_score

In [ ]:
"""
Correlate TPSim with behavioral variables // Memory group
BY ELECTRODE and Plot summary (for all included electrodes)
"""

exps, RT_type = ['Enc'], 'mem_groups' #RT_rec_first, RT_rec_wght, RT_epi_wght, RT_epi_first
freqs = ['theta'] #'1_alpha', '2_beta','3_gamma', high_gamma
meth, conds, stat = 'btw', ['low','mid','high'], 'LinReg'
subjects = ['CHAF','VACJ','PIRJ','SEMC','FERJ','LEFC']
rois_sel = ['aHC','MFG','ACC','IFG','Amg','pPirT','PHG','Ins_olf',
            'OFC_olf','SFG','HC']
width, step = 'None', 'None'
st = study('Olfacto')

for exp in exps:
    ###############################################################################
    path_tps = join(st.path, 'feature/TPSim_3groups_'+exp+'/')
    tps_form = join(path_tps, 'TPS_'+meth+'_v=1_elecs=all_hilbert=ok/'+\
                        'TPS_pears_{}_{}_'+meth+'_{}_width={}_step={}_ds.npz')
    df_path = join(path_tps, 'LinReg_stats_{}_v=1_elecs=all_hilbert=ok/')
    df_name = join(df_path, '{}_ols_'+meth+'_{}_{}_{}_width={}_step={}_ds.csv') #su, conds0, conds1, freq
    ###############################################################################
    
    dict_perf = {'low':1, 'mid':2, 'high':3}
    #dict_perf = {'low':1, 'high':2}

    for freq in freqs:
        if not exists(df_path.format(freq)):
            makedirs(df_path.format(freq))
        subjects_c, elecs_c, labels_c = np.array([]), np.array([]), np.array([])
        channels_c, x_c, y_c, z_c = np.array([]), np.array([]), np.array([]), np.array([])
        tps_scores, T_vals_c, p_vals_c = np.array([]), np.array([]), np.array([])
        p_fdr_c, p_bf_c = np.array([]), np.array([])
        
        for su in subjects:
            #load all elec info,rename and select electrodes id
            mat = np.load(tps_form.format(su,conds[0],freq,width,step),allow_pickle=True)
            labels, channels = mat['label'], mat['channel']
            print(su,labels.shape, mat['tps'].shape,np.unique(labels))
            x, y, z = mat['xyz'][:,0], mat['xyz'][:,1], mat['xyz'][:,2]
            labels_new = rename_elecs(labels,x,y,z)
            idx_sel = [i for i,lab in enumerate(labels_new) if lab in rois_sel]
            labels, channels = labels_new[idx_sel], channels[idx_sel]
            x, y, z = x[idx_sel], y[idx_sel], z[idx_sel]
            
            if stat == 'LinReg':
                tps_su = np.zeros((len(idx_sel),len(conds)))
                all_tps, all_scores = [], []
                for c,cond in enumerate(conds):
                    mat = np.load(tps_form.format(su,cond,freq,width,step))
                    tps = mat['tps'][idx_sel]
                    nelecs, ntrials = tps.shape
                    score_ = np.array([dict_perf[cond]]*ntrials)
                    tps_su[:,c] += np.mean(tps,axis=1)
                    #fill tps and score vectors
                    all_tps.append(tps)
                    all_scores.append(score_)

                #compute stats Ttests-unpaired
                all_tps = np.concatenate(all_tps,axis=1)
                all_scores = np.concatenate(all_scores,axis=0)
                T, unc_p = [], []
                for elec in range(nelecs):
                    #Tval,pval = stats.kendalltau(all_tps[elec],all_scores)
                    Y, X = np.array(all_tps[elec]), sm.add_constant(np.array(all_scores))
                    model_ols = sm.OLS(Y,X).fit()
                    Tval, pval = np.round(model_ols.tvalues[1],3),model_ols.pvalues[1]
                    T.append(Tval), unc_p.append(pval)
            if stat == 'Ttest':
                tps0 = np.load(tps_form.format(su,conds[0],freq,width,step))['tps'][idx_sel]
                nelecs = len(idx_sel)
                tps1 = np.load(tps_form.format(su,conds[1],freq,width,step))['tps'][idx_sel]
                mean0, mean1 = np.mean(tps0,axis=1)[:,np.newaxis], np.mean(tps1,axis=1)[:,np.newaxis]
                tps_su = np.concatenate((mean0,mean1),axis=1)
                #compute stats Ttests-unpaired
                tps0, tps1 = tps0.swapaxes(0,1), tps1.swapaxes(0,1) #ntrials x nelecs
                T, unc_p = ttest_ind(tps0, tps1, equal_var=False)
                print(T.shape, unc_p.shape)
            _, p_fdr = fdr_correction(unc_p)
            _, p_bf = bonferroni_correction(unc_p)

            #fill all df data
            subjects_c = np.hstack((subjects_c,np.array([su]*nelecs))) if np.size(subjects_c) else np.array([su]*nelecs)
            elecs_c = np.hstack((elecs_c,np.arange(nelecs))) if np.size(elecs_c) else np.arange(nelecs)
            labels_c = np.hstack((labels_c,labels)) if np.size(labels_c) else labels
            channels_c = np.hstack((channels_c,channels)) if np.size(channels_c) else channels
            tps_scores = np.concatenate((tps_scores,tps_su),axis=0) if np.size(tps_scores) else tps_su
            x_c = np.hstack((x_c,x)) if np.size(x_c) else x
            y_c = np.hstack((y_c,y)) if np.size(y_c) else y
            z_c = np.hstack((z_c,z)) if np.size(z_c) else z
            T_vals_c = np.hstack((T_vals_c,T)) if np.size(T_vals_c) else T
            p_vals_c = np.hstack((p_vals_c,unc_p)) if np.size(p_vals_c) else unc_p
            p_fdr_c = np.hstack((p_fdr_c,p_fdr)) if np.size(p_fdr_c) else p_fdr
            p_bf_c = np.hstack((p_bf_c,p_bf)) if np.size(p_bf_c) else p_bf

        data = np.concatenate((subjects_c[:,np.newaxis],labels_c[:,np.newaxis],
                    channels_c[:,np.newaxis],x_c[:,np.newaxis],y_c[:,np.newaxis],
                    z_c[:,np.newaxis],elecs_c[:,np.newaxis],tps_scores,T_vals_c[:,np.newaxis],
                    p_vals_c[:,np.newaxis],p_fdr_c[:,np.newaxis],p_bf_c[:,np.newaxis]),
                    axis=1)
        df = pd.DataFrame(data, columns=['subjects','labels','channels','x','y','z',
                                'elecs_num', 'tps_'+conds[0], 'tps_'+conds[1],'tps_'+conds[2],
                                        'Tvals', 'unc_p','fdr_p', 'bonf_p'])
        print(df.shape)
        df.to_csv(df_name.format(freq,'All_subjects',freq,RT_type,stat,width,step),index=False)   

In [ ]:
"""Create a big csv file with all Freesurfer labels"""

PATH = '/media/karim/Datas4To/1_Analyses_Intra_EM_Odor/Implantation_Patients_MNI/db_slicer_labels/'
file_name = '/{}_radius-5.0_hip-hires.xlsx'
PATH_SAVE = join(st.path, 'feature/')
subjects = ['S04_FERJ','S02_SEMC','S03_LEFC','S01_VACJ','S05_PIRJ','S06_MICP','S00_CHAF']
savename = 'All_subjects_Fressurfer_labels.csv'

#concat all labels together
df_tot = pd.DataFrame()
for su in subjects:
    df = pd.read_excel(PATH+su+file_name.format(su), sheet_name='bipolar')
    nelecs = df.shape[0]
    df_sel = df[['contact','Freesurfer']]
    df_sel['subjects'] = [su[4:]]*nelecs
    df_sel['hip_CA'] = [lab0 if lab0 != 'none' else lab1 for lab0,lab1 in \
                                zip(df['lh.hippoAmygLabels-T1.v21.CA.FSvoxelSpace'],
                                    df['rh.hippoAmygLabels-T1.v21.CA.FSvoxelSpace'])]
    df_sel['hip_FS60'] = [lab0 if lab0 != 'none' else lab1 for lab0,lab1 in \
                                zip(df['lh.hippoAmygLabels-T1.v21.FS60.FSvoxelSpace'],
                                    df['rh.hippoAmygLabels-T1.v21.FS60.FSvoxelSpace'])]
    columnsTitles = ['subjects', 'contact', 'Freesurfer','hip_CA','hip_FS60']
    df_sel = df_sel.reindex(columns=columnsTitles)
    df_tot = df_tot.append(df_sel)
df_tot.to_csv(PATH_SAVE+savename)

In [ ]:
"""Add freesurfer labels to the results df"""

###############################################################################
path_tps = join(st.path, 'feature/')
df_path = join(path_tps, 'TPSim_3groups_'+exp+'/LinReg_stats_{}_v=1_elecs=all_hilbert=ok/')
# df_path = join(path_tps, 'TPSim_3groups_'+exp+'/Ttest_odor_identity_v=1_elecs=all/')
df_name = join(df_path, '{}_ols_'+meth+'_{}_{}_{}.csv') #su, conds0, conds1, freq
# df_name = join(df_path, 'Bilan_All_subjects_Ttests_Low_High_all_roi_aHC_fdr_0.05_theta.csv') #su, conds0, conds1, freq
df_f_name = join(path_tps, 'All_subjects_Fressurfer_labels.csv')
###############################################################################
exp, freq, meth, stat = 'Enc', 'theta', 'btw', 'LinReg'

df = pd.read_csv(df_name.format(freq,'All_subjects',freq, 'mem_groups',stat))
# df = pd.read_csv(df_name)
_, su_idx = np.unique(df['subjects'].values, return_index=True)
subjects = df['subjects'].values[np.sort(su_idx)]
df_f = pd.read_csv(df_f_name)

free_labs, hip_CA, hip_fs60 = [], [], []
for su in subjects:
    #select one subject's df and select only good contacts labels
    df_su = df.loc[df['subjects']==su]
    df_f_su = df_f.loc[df_f['subjects']==su]
    list_chans = [lab.replace("'","") for lab in df_su['channels'].str.upper()] \
                                if su == 'CHAF' else df_su['channels'].str.upper().values
    idx = [i for i,lab in enumerate(df_f_su['contact'].values) if lab in list_chans]
    
    #due to contact type 3x5 some are missing in Freesurfer data
    missing = [elec for elec in list_chans if elec not in df_f_su['contact'].values[idx]]
    idx_miss = [i for i,elec in enumerate(list_chans) if elec in missing]
    idx_miss = [ind -i for i, ind in enumerate(idx_miss)]
    print(su, 'missing contacts', missing, idx_miss)
    
    #select all subject's labels
    fr_l = df_f_su['Freesurfer'].values[idx]
    ca_l = df_f_su['hip_CA'].values[idx]
    fs_l = df_f_su['hip_FS60'].values[idx]
    
    #add None labels at the good index location in subject's df
    fr_l = np.insert(fr_l, idx_miss, 'not_fs')
    ca_l = np.insert(ca_l, idx_miss, 'not_fs')
    fs_l = np.insert(fs_l, idx_miss, 'not_fs')
    free_labs.extend(fr_l), hip_CA.extend(ca_l), hip_fs60.extend(fs_l)

df['fsf'], df['hip_CA'], df['hip_FS60'] = free_labs, hip_CA, hip_fs60
#save new files with all labels
df_save = df_name.format(freq,'All_subjects',freq, 'mem_groups',stat).replace('.csv','_fsf.csv')
df.to_csv(df_save,index=False)

In [ ]:
from collections import Counter

st = study('Olfacto')
exp, stat = 'Enc', 'LinReg'#'Enc'
meth, RT_types = 'btw', ['mem_groups']#['RT_epi_first','RT_epi_wght','RT_rec_first','RT_rec_wght']
olf_regions = ['Amg','pPirT','OFC_olf']
freq, width, step = 'theta', 'None', 'None'
thrs = [0.05]
corrections = ['fdr_p']#['fdr_p'] unc_p

##################################################################################
path_pow = join(st.path, 'feature/TPSim_3groups_'+exp+'/LinReg_stats_{}_v=1_elecs=all_hilbert=ok/')
df_name = join(path_pow, '{}_ols_'+meth+'_{}_{}_{}_width={}_step={}_ds.csv') #su, conds0, conds1, freq
df_stat_save = join(path_pow, 'Bilan_{}_OLS_'+meth+'_{}_{}_{}_{}_{}_{}_width={}_step={}_ds.csv')
df_stat_all = join(path_pow, 'Bilan_{}_OLS_'+meth+'_{}_{}_{}_width={}_step={}_ds.csv')
##################################################################################

for RT_type in RT_types:
    df = pd.read_csv(df_name.format(freq,'All_subjects',freq,RT_type,stat,width,step))
    df['labels'] = ['pPirT' if lab == 'Amg' else lab for lab in df['labels']]
    print('Initial df shape', df.shape, meth, exp, RT_type, stat)
    for th, corr in product(thrs,corrections):
        df_sel = df.loc[df[corr]<th] #
        print(df.loc[df['fdr_p']<th])
        df_sel['sign'] = ['separation' if t > 0 else 'completion' for t in df_sel['Tvals']]
        print('\n',meth,'stats at p < ',th, 'correction : ',corr, df_sel.shape)
        df_sel.to_csv(df_stat_all.format(freq,'All_subjects',freq,RT_type,
                                                 corr+str(th),width,step))
        #print(df_sel.groupby(['fsf','subjects','sign']).count())
        rois = np.unique(df_sel['labels'])
        for roi in rois:
            df_roi = df_sel.loc[df_sel['labels']==roi]
            df_inc = df_roi.loc[df_roi['sign']=='completion'].groupby(['subjects']).count()
            df_dec = df_roi.loc[df_roi['sign']=='separation'].groupby(['subjects']).count()
            #print(roi, 'integration', df_inc.shape, 'segregation',df_dec.shape)
            
            if (df_inc.shape[0] >= 3) or (df_inc.shape[0] >=2 and roi in olf_regions):
                print(roi, 'NB of subjects with completion',df_inc.shape[0],' subjects')
                df_plot = df_roi.loc[df_roi['sign']=='completion']
                print('#electrodes in total >>> ',df_plot.shape[0])
                df_plot.to_csv(df_stat_save.format(freq,'All_subjects',freq,'mem_groups',
                                                   'compl',roi,corr+str(th),stat,width,step))

            if (df_dec.shape[0] >= 3) or (df_dec.shape[0] >=2 and roi in olf_regions):
                print(roi, 'NB of subjects with separation',df_dec.shape[0],' subjects')
                df_plot = df_roi.loc[df_roi['sign']=='separation']
                print('#electrodes in total >>> ',df_plot.shape[0])
                df_plot.to_csv(df_stat_save.format(freq,'All_subjects','theta','mem_groups',
                                                   'sep',roi,corr+str(th),stat,width,step))

### Bar plots for sig electrodes

In [ ]:
import matplotlib.pyplot as plt
st = study('Olfacto')
PATH = join(st.path, 'feature/TPSim_3groups_{}/LinReg_stats_{}_v=1_elecs=all/')
# PATH = join(st.path, 'feature/TPSim_3groups_{}/Ttest_odor_identity_v=1_elecs=all/')
SAVE_PATH = join(PATH, 'Nb_elecs_{}_by_subject.png')
meth, exp, freq= 'btw', 'Enc_Ret', 'theta'

files = st.search('Bilan_All_subjects_',folder=PATH.format(exp,freq))
#files = st.search('0.05_'+freq,folder=PATH.format(exp,freq))

for f in files:
    splits = f.split('_')[-5:]
    f_name = splits[0]+'_'+splits[1]+'_'+splits[2]+'_'+splits[3]
    df = pd.read_csv(PATH.format(exp,freq)+f)
    
    #Plot nb of significant electrodes by win and subjects
    colors = {'CHAF':'darkblue', 'FERJ':'royalblue', 'LEFC':'deepskyblue', 
            'PIRJ':'yellow','SEMC':'darkorange', 'VACJ':'red'}
    df1 = df[['subjects','labels']].groupby(['subjects']).count()
    xticks, w = np.arange(0,1,0.2), 0.5
    fig = plt.figure()
    bottom = np.zeros(1)
    for i,su in enumerate(df1.index):
        count = df1.iloc[[i]].values[0]
        xpts = range(len(count))
        print('shape to plot',count.shape, xticks.shape)
        plt.bar(xpts, count, color=colors[su], label=su, bottom=bottom)
        bottom += count
    plt.title('Significant electrodes for '+freq)
    plt.ylabel('Nb of electrodes')
    plt.xticks(xticks,freq)
    plt.ylim(0,15)
    plt.legend(loc=0,handletextpad=0.1, frameon=False)
    plt.savefig(SAVE_PATH.format(exp,freq,f_name))
    plt.savefig(SAVE_PATH.format(exp,freq,f_name).replace('.png','.pdf'))
    plt.clf()
    plt.close()

### Plot average sig data

In [ ]:
import matplotlib.pyplot as plt

PATH = join(st.path, 'feature/TPSim_3groups_{}/LinReg_stats_{}_v=1_elecs=all_hilbert=ok/')
#PATH = join(st.path, 'feature/TPSim_3groups_{}/TPS_btw_thgh_time_v=1_elecs=all/')
# PATH = join(st.path, 'feature/TPSim_3groups_{}/Ttest_odor_identity_v=1_elecs=all/')
SAVE_PATH = join(PATH, 'Mean_{}_{}_tps_by_subject.png')
meth, exp, freq= 'btw', 'Enc', 'theta'
width, step = '256', '51'
conds = ['low','mid','high']
#conds = ['early','late']
# conds = ['tps_wth','tps_btw','wth-btw','tps_diff']
features = ['Tvals','tps']

files = st.search('Bilan_All_subjects_OLS_',folder=PATH.format(exp,freq))
#files = st.search('0.05_'+freq,folder=PATH.format(exp,freq))

for f in files:
    splits = f.split('_')[-5:]
    f_name = splits[0]+'_'+splits[1]+'_'+splits[2]+'_'+splits[3]
    df = pd.read_csv(PATH.format(exp,freq)+f)
    df_gr = df #df.groupby(['subjects']).mean()
    
    for feat in features:
        df_f = df_gr.filter(like=feat)
        mean, sem = df_f.mean(), df_f.sem()
        print(f_name, feat, mean,sem)
        #Plot Mean AUC Score
        fig = plt.figure()
        plt.title('Mean {} Score by region for {}'.format(feat,freq))
        plt.ylabel('Mean '+feat)
        xticks, w = np.arange(0,df_f.shape[1]), 0.8
        plt.xticks(xticks,df_f.columns)
        plt.bar(xticks,mean,yerr=sem,color='darkorange')
        
        plt.savefig(SAVE_PATH.format(exp,freq,feat,f_name))
        plt.savefig(SAVE_PATH.format(exp,freq,feat,f_name).replace('.png','.pdf'))
        plt.clf()
        plt.close()
    
    

#### Plot individual data with patient name and electrode number

In [ ]:
from brainpipe.system import study
import matplotlib.pyplot as plt
from os.path import join
import pandas as pd
import numpy as np

st = study('Olfacto')
#PATH = join(st.path, 'feature/TPSim_3groups_{}/LinReg_stats_{}_v=1_elecs=all/')
PATH = join(st.path, 'feature/TPSim_3groups_{}/Ttest_odor_identity_v=1_elecs=all/')
SAVE_PATH = join(PATH, 'Ind_elecs_{}_{}_avg_subject_3gr_LinReg.png')
meth, exp, freq= 'btw', 'Enc', 'theta'
features = ['tps_low','tps_mid','tps_high']
features_sd = ['sem_low','sem_mid','sem_high']

files = st.search('Bilan_All_subjects_LinReg_',folder=PATH.format(exp,freq))

for f in files:
    splits = f.split('_')[-6:]
    if '3gr' in splits:
        print('selected',f)
        f_name = splits[1]+'_'+splits[2]+'_'+splits[3]+'_'+splits[4]
        df = pd.read_csv(PATH.format(exp,freq)+f)
        df_m = df.groupby(['subjects']).agg(['mean','sem'])
        nelecs = df_m.shape[0]

        fig, axs = plt.subplots(2,4, figsize=(15, 6), facecolor='w', edgecolor='k')
        fig.subplots_adjust(hspace = .5, wspace=.001)

        axs = axs.ravel()

        for i in range(nelecs):
            df_sel = df_m.iloc[i,:].unstack().T
            #print(df_sel)

            su, roi =  df_m.index[i], splits[1]
            data = df_sel[features].iloc[0,:].values
    #         yerr = df_sd.iloc[i,:][features].values
            yerr = df_sel[features].iloc[1,:].values
            #print(data,yerr)

            xticks, w = np.arange((len(features))), 0.8
            axs[i].set_xticks(xticks,features)
            axs[i].bar(xticks,data,yerr=yerr,color='blue')
            axs[i].set_title('{} in {}'.format(su,roi))
        plt.setp(axs, ylim=(0,1))
        plt.savefig(SAVE_PATH.format(exp,'tps',f_name))
        plt.savefig(SAVE_PATH.format(exp,'tps',f_name).replace('.png','.pdf'))
        plt.clf()
        plt.close()
    

### Plot regression plots for sig elecs with all data

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.offsetbox import AnchoredText
from scipy.stats import sem

###############################################################################
st = study('Olfacto')
path_tps = join(st.path, 'feature/TPSim_3groups_{}/')
tps_form = join(path_tps, 'TPS_btw_v=1_elecs=all/TPS_pears_{}_{}_btw_{}.npz')
df_path = join(path_tps, 'LinReg_stats_{}_v=1_elecs=all/')
SAVE_PATH = join(df_path, 'LinReg_{}_{}_avg_subject.png')
###############################################################################
freq = 'theta' #'1_alpha', '2_beta','3_gamma', high_gamma
meth, exp, conds = 'btw', 'Enc_Ret', ['low','mid','high']
subjects = ['CHAF','VACJ','PIRJ','SEMC','FERJ','LEFC']
dict_perf = {'low':1, 'mid':2, 'high':3}

files = st.search('Bilan_All_subjects_OLS_btw_'+freq,folder=df_path.format(exp,freq))

for fi in files:
    splits = f.split('_')[-5:]
    f_name = splits[0]+'_'+splits[1]+'_'+splits[2]+'_'+splits[3]
    df = pd.read_csv(df_path.format(exp,freq)+fi)
    subjects = np.unique(df[['subjects']])
    
    tps_su = np.zeros((len(subjects),len(conds)))
    for s,su in enumerate(subjects):
        df_su = df.loc[df['subjects']==su]
        chans_sig = df_su[['channels']].values
        
        mat = np.load(tps_form.format(exp,su,conds[0],freq),allow_pickle=True)
        channels = mat['channel']
        id_chans = [i for i,chan in enumerate(channels) if chan in chans_sig]
            
        all_tps, all_scores = [], []
        tps_cds, sem_cds = [], []
        for c,cond in enumerate(conds):
            mat = np.load(tps_form.format(exp,su,cond,freq))
            tps = mat['tps'][id_chans]
            nelecs, ntrials = tps.shape
            score_ = np.array([dict_perf[cond]]*ntrials)
            all_tps.append(tps)
            all_scores.append(score_)
            tps_cds.append(np.mean(tps))
            sem_cds.append(sem(np.mean(tps,axis=0)))
        
        #compute stats Ttests-unpaired
        tps_su[s] += tps_cds
        all_tps = np.concatenate(all_tps,axis=1)
        all_scores = np.concatenate(all_scores,axis=0)
        all_T, all_p = [], []
        for elec in range(nelecs):
            Y, X = np.array(all_tps[elec]), sm.add_constant(np.array(all_scores))
            model_ols = sm.OLS(Y,X).fit()
            #print(model_ols.summary())
            Tval, pval = np.round(model_ols.tvalues[1],2),np.round(model_ols.pvalues[1],3)
            all_T.append(Tval), all_p.append(pval)
        #Plot all values by subject
        tps_plot = np.mean(all_tps,axis=0)
        T, p = all_T[np.argmin(abs(np.array(all_T)))],np.max(np.array(all_p))
        fig, ax = plt.subplots(figsize=(3,3))
        #ax = sns.stripplot(x=all_scores, y=tps_plot, alpha=.25, zorder=1)
        ax = sns.violinplot(x=all_scores, y=tps_plot, alpha=.25, zorder=1, bw=0.5,scale='width')
        ax = sns.pointplot(x=all_scores, y =tps_plot, markers="d", scale=.75,
                          estimator=np.median, ci=None)
        #plt.errorbar([0,1,2],tps_cds,yerr=sem_cds,fmt='o')
        #m, b = np.polyfit(all_scores, tps_plot, 1)
        #plt.plot(all_scores-1, (m*all_scores)+b)
        if len(id_chans) == 1:
            anchored_text = AnchoredText('t = %s \np = %s' % (T, p),loc=2)
        elif len(id_chans) > 1 and T > 0:
            anchored_text = AnchoredText('t > %s \np < %s' % (T, p),loc=2)
        elif len(id_chans) > 1 and T < 0:
            anchored_text = AnchoredText('t < %s \np < %s' % (T, p),loc=2)
        ax.add_artist(anchored_text)
        plt.savefig(SAVE_PATH.format(exp,freq,su,f_name))
        plt.savefig(SAVE_PATH.format(exp,freq,su,f_name).replace('png','pdf'))
            

#### Linear Regression analyses for electrodes significant at encoding

In [ ]:
"""
Linear Regression of TPD with memory richness
Only subjects and ROIs with significant results at Encoding
Take SIGNIFICANT electrodes within ROI and FDR correct across subjects
"""

import statsmodels.api as sm
import statsmodels.formula.api as smf
from os import path
import pingouin as pg
from utils import odor_groups_3wgth
from scipy import stats
import math

############################################################################################
st = study('Olfacto')
fold, fold2, freq, suffix = 'Enc','Enc_Ret','theta',  'E=E_R=all'
path_df = path.join(st.path, 'feature/TPSim_3groups_{}/LinReg_stats_theta_v=1_elecs=all/')
path_data = path.join(st.path, 'feature/TPSim_3groups_'+fold2+'/TPS_btw_v=1_elecs=all/')
filename = path.join(path_data, 'TPS_pears_{}_{}_btw_theta_'+suffix+'.npz')
############################################################################################
corr = 'fdr'
conds = ['low','mid','high']
files = st.search('Bilan_All_subjects_OLS',
              folder='feature/TPSim_3groups_'+fold+'/LinReg_stats_theta_v=1_elecs=all/')
dict_score = {'low':0, 'mid':1, 'high':2}

for fi in files:
    if corr in fi.split('_'):
        roi = fi.split('_')[9]
        roi = 'OFC_olf' if roi == 'OFC' else roi
        #if roi in ['OFC','aHC','Amg']:
        print('>>>> processing ',fi, roi)
        df = pd.read_csv(path_df.format(fold)+fi)
        _, idx = np.unique(df['subjects'], return_index=True)
        subjects = df['subjects'][np.sort(idx)]
        print('>>>> {} subjects significant'.format(len(subjects)))
        print(subjects)
        pvals, Tvals, subj, chans = [], [], [], []
        tps_low, tps_high, sem_low, sem_high = [], [], [], []
        tps_mid, sem_mid = [], []
        for su in subjects:
            df_su = df.loc[df['subjects']==su]
            chans_df = df_su['channels'].values

            all_su_tps, all_scores = [], []
            for cond in conds:
                mat = np.load(filename.format(su,cond),allow_pickle=True)
                idx_elecs = [i for i,chan in enumerate(mat['channel']) if chan in chans_df]
                
                #if roi != 'Amg':
                #    idx_elecs = [i for i,lab in enumerate(mat['label']) if lab == roi]
                #if roi == 'Amg':
                #    idx_elecs = [i for i,lab in enumerate(mat['label']) if lab in ['Amg','pPirT']]
                channels = mat['channel'][idx_elecs]
                data = mat['tps'][idx_elecs,:]
                nelecs, ntrials = data.shape
                all_su_tps.append(data)
                score_ = np.array([dict_score[cond]]*ntrials)
                all_scores.append(score_)
            
            print([x.shape for x in all_su_tps])
            if all_su_tps[-1].shape[-1] > 1: #pass subjects without enough trials
                all_tps_concat = np.concatenate(all_su_tps,axis=1)
                all_scores_concat = np.concatenate(all_scores,axis=0)
                T, unc_p = [], []
                for elec in range(nelecs):
                    Y = np.array(all_tps_concat[elec])
                    X = sm.add_constant(np.array(all_scores_concat))
                    model_ols = sm.OLS(Y,X).fit()
                    Tval, pval = np.round(model_ols.tvalues[1],3),model_ols.pvalues[1]
                    Tvals.append(Tval), pvals.append(pval)
    
                    subj.append(su), chans.append(channels[elec])
                    tps_low.append(np.mean(all_su_tps[0][elec]))
                    tps_mid.append(np.mean(all_su_tps[1][elec]))
                    tps_high.append(np.mean(all_su_tps[2][elec]))
                    sem_low.append(stats.sem(all_su_tps[0][elec]))
                    sem_mid.append(stats.sem(all_su_tps[1][elec]))
                    sem_high.append(stats.sem(all_su_tps[2][elec]))

        _, pvals_fdr = fdr_correction(pvals)
        _, pvals_bonf = bonferroni_correction(pvals)
        
        subj, chans = np.array(subj),np.array(chans)
        tps_low, tps_high, Tvals = np.array(tps_low),np.array(tps_high), np.array(Tvals)
        sem_low, sem_high, pvals = np.array(sem_low),np.array(sem_high),np.array(pvals)
        sem_mid, tps_mid = np.array(sem_mid),np.array(tps_mid)
        #print(pvals.shape, pvals_fdr.shape,pvals_bonf.shape, Tvals.shape)
        data_final = np.concatenate([subj[:,np.newaxis],chans[:,np.newaxis],
                tps_low[:,np.newaxis], sem_low[:,np.newaxis],
                tps_mid[:,np.newaxis], sem_mid[:,np.newaxis],
                tps_high[:,np.newaxis],sem_high[:,np.newaxis],
                Tvals[:,np.newaxis],pvals[:,np.newaxis],
                pvals_fdr[:,np.newaxis],pvals_bonf[:,np.newaxis]],axis=1)
        
        df = pd.DataFrame(data_final, columns=['subjects','channels',
                    'tps_low', 'sem_low', 'tps_mid', 'sem_mid','tps_high', 'sem_high', 
                    'Tvals_diff','unc_diff','fdr_diff','bonf_diff'])
        
        df['fdr_diff'] = df['fdr_diff'].astype(float)
        df['Tvals_diff'] = df['Tvals_diff'].astype(float)
        df_sig = df.loc[(df['fdr_diff']<0.05)&(df['Tvals_diff']<0)]
        if df_sig.shape[0] > 0:
            print(df_sig)
        df.to_csv(path_df.format(fold2)+fi.replace('mem_groups',
                                        'elecsE_3gr').replace('.csv',suffix+'.csv'))

### SAME analysis but with 2 classes

In [ ]:
"""
LOW vs HIGH TPD at Retrieval and Reinstatement
Only subjects and ROIs with significant results at Encoding
Take SIGNIFICANT electrodes within ROI and FDR correct across subjects
"""

import statsmodels.api as sm
import statsmodels.formula.api as smf
from os import path
import pingouin as pg
from utils import odor_groups_3wgth
from scipy import stats
import math

############################################################################################
st = study('Olfacto')
fold, fold2, freq = 'Enc','Enc_Ret','theta'
path_df = path.join(st.path, 'feature/TPSim_3groups_{}/LinReg_stats_theta_v=1_elecs=all/')
path_data = path.join(st.path, 'feature/TPSim_3groups_'+fold2+'/TPS_btw_v=1_elecs=all/')
filename = path.join(path_data, 'TPS_pears_{}_{}_btw_theta.npz')
############################################################################################
corr = 'fdr'
conds = ['low','mid','high']
files = st.search('Bilan_All_subjects_OLS',
              folder='feature/TPSim_3groups_'+fold+'/LinReg_stats_theta_v=1_elecs=all/')
dict_score = {'low':0, 'mid':1, 'high':2}

for fi in files:
    if corr in fi.split('_'):
        roi = fi.split('_')[9]
        roi = 'OFC_olf' if roi == 'OFC' else roi
        print('>>>> processing ',fi, roi)
        df = pd.read_csv(path_df.format(fold)+fi)
        _, idx = np.unique(df['subjects'], return_index=True)
        subjects = df['subjects'][np.sort(idx)]
        print('>>>> {} subjects significant'.format(len(subjects)))
        print(subjects)
        pvals, Tvals, subj, chans = [], [], [], []
        tps_low, tps_high, sem_low, sem_high = [], [], [], []
        tps_mid, sem_mid = [], []
        for su in subjects:
            df_su = df.loc[df['subjects']==su]
            chans_df = df_su['channels'].values
            all_su_tps = []
            for cond in conds:
                mat = np.load(filename.format(su,cond),allow_pickle=True)
                idx_elecs = [i for i,chan in enumerate(mat['channel']) if chan in chans_df]
                
                #if roi != 'Amg':
                #    idx = [i for i,lab in enumerate(mat['label']) if lab == roi]
                #if roi == 'Amg':
                #    idx = [i for i,lab in enumerate(mat['label']) if lab in ['Amg','pPirT']]
                channels = mat['channel'][idx_elecs]
                data = mat['tps'][idx_elecs,:]
                nelecs = data.shape[0]
                all_su_tps.append(data)

            if all_su_tps[-1].shape[-1] > 1: #pass subjects without enough trials
                print(su, [x.shape for x in all_su_tps])    
                for elec in range(nelecs):
                    df_ = pd.DataFrame()
                    df_['tps'] = np.concatenate([x[elec] for x in all_su_tps])
                    df_['mem_score'] = np.concatenate([[cond]*x.shape[1] for cond,x in zip(conds,all_su_tps)])
                    df_['mem_val'] = df_['mem_score'].map(dict_score)
                    df_ = df_.loc[df_['mem_score']!='mid']

                    ttests = pg.pairwise_gameshowell(data=df_, dv='tps',
                                                    tail='one-sided',between='mem_score') 
                    stats_ = np.round(ttests['T'].values,3),np.round(ttests['pval'].values,3)
                    pvals.append(stats_[1]), Tvals.append(stats_[0])
                    subj.append(su), chans.append(channels[elec])
                    tps_low.append(np.mean(all_su_tps[0][elec]))
                    tps_mid.append(np.mean(all_su_tps[1][elec]))
                    tps_high.append(np.mean(all_su_tps[2][elec]))
                    sem_low.append(stats.sem(all_su_tps[0][elec]))
                    sem_mid.append(stats.sem(all_su_tps[1][elec]))
                    sem_high.append(stats.sem(all_su_tps[2][elec]))
                    
        pvals = [p if not math.isnan(p) else 1 for p in pvals]
        _, pvals_fdr = fdr_correction(pvals)
        _, pvals_bonf = bonferroni_correction(pvals)
        
        subj, chans = np.array(subj),np.array(chans)
        tps_low, tps_high, Tvals = np.array(tps_low),np.array(tps_high), np.array(Tvals)
        sem_low, sem_high, pvals = np.array(sem_low),np.array(sem_high),np.array(pvals)
        sem_mid, tps_mid = np.array(sem_mid),np.array(tps_mid)
        
        data_final = np.concatenate([subj[:,np.newaxis],chans[:,np.newaxis],
                tps_low[:,np.newaxis], sem_low[:,np.newaxis],
                tps_mid[:,np.newaxis], sem_mid[:,np.newaxis],
                tps_high[:,np.newaxis],sem_high[:,np.newaxis],
                Tvals,pvals,pvals_fdr,pvals_bonf],axis=1)
        
        df = pd.DataFrame(data_final, columns=['subjects','channels',
                    'tps_low', 'sem_low', 'tps_mid', 'sem_mid','tps_high', 'sem_high', 
                    'Tvals_diff','unc_diff','fdr_diff','bonf_diff'])
        
        df['fdr_diff'] = df['fdr_diff'].astype(float)
        df['Tvals_diff'] = df['Tvals_diff'].astype(float)
        df.to_csv(path_df.format(fold2)+fi.replace('mem_groups','elecsE_2gr'))

### Plot individual violin plots for NON significant electrodes

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.offsetbox import AnchoredText
from scipy.stats import sem

###############################################################################
st = study('Olfacto')
path_tps = join(st.path, 'feature/TPSim_3groups_{}/')
tps_form = join(path_tps, 'TPS_btw_v=1_elecs=all/TPS_pears_{}_{}_btw_{}.npz')
df_path = join(path_tps, 'LinReg_stats_{}_v=1_elecs=all/')
SAVE_PATH = join(df_path, 'LinReg_{}_{}_NOT_sig.png')
###############################################################################
freq = 'theta' #'1_alpha', '2_beta','3_gamma', high_gamma
meth, exp, conds = 'btw', 'Enc', ['low','mid','high']
subjects = ['CHAF','VACJ','PIRJ','SEMC','FERJ','LEFC']
dict_perf = {'low':1, 'mid':2, 'high':3}
rois = ['aHC','OFC_olf']

for roi in rois:
    for s,su in enumerate(subjects):
        mat = np.load(tps_form.format(exp,su,conds[0],freq),allow_pickle=True)
        labels = mat['label']
        id_labs = [i for i,lab in enumerate(labels) if lab == roi]
        
        if len(id_labs) > 0:
            all_tps, all_scores = [], []
            tps_cds, sem_cds = [], []
            for c,cond in enumerate(conds):
                mat = np.load(tps_form.format(exp,su,cond,freq))
                tps = mat['tps'][id_labs]
                nelecs, ntrials = tps.shape
                score_ = np.array([dict_perf[cond]]*ntrials)
                all_tps.append(tps)
                all_scores.append(score_)
                tps_cds.append(np.mean(tps))
                sem_cds.append(sem(np.mean(tps,axis=0)))

            #compute stats Ttests-unpaired
            all_tps = np.concatenate(all_tps,axis=1)
            all_scores = np.concatenate(all_scores,axis=0)
            all_T, all_p = [], []
            for elec in range(nelecs):
                Y, X = np.array(all_tps[elec]), sm.add_constant(np.array(all_scores))
                model_ols = sm.OLS(Y,X).fit()
                #print(model_ols.summary())
                Tval, pval = np.round(model_ols.tvalues[1],2),np.round(model_ols.pvalues[1],3)
                all_T.append(Tval), all_p.append(pval)
            #Plot all values by subject
            tps_plot = np.mean(all_tps,axis=0)
            T, p = all_T[np.argmin(abs(np.array(all_T)))],np.max(np.array(all_p))
            fig, ax = plt.subplots(figsize=(3,3))
            #ax = sns.stripplot(x=all_scores, y=tps_plot, alpha=.25, zorder=1)
            ax = sns.violinplot(x=all_scores, y=tps_plot, alpha=.25, zorder=1, bw=0.5,scale='width')
            ax = sns.pointplot(x=all_scores, y =tps_plot, markers="d", scale=.75,
                              estimator=np.median, ci=None)
            #plt.errorbar([0,1,2],tps_cds,yerr=sem_cds,fmt='o')
            #m, b = np.polyfit(all_scores, tps_plot, 1)
            #plt.plot(all_scores-1, (m*all_scores)+b)
            if len(id_labs) == 1:
                anchored_text = AnchoredText('t = %s \np = %s' % (T, p),loc=2)
            elif len(id_labs) > 1 and T > 0:
                anchored_text = AnchoredText('t > %s \np < %s' % (T, p),loc=2)
            elif len(id_labs) > 1 and T < 0:
                anchored_text = AnchoredText('t < %s \np < %s' % (T, p),loc=2)
            ax.add_artist(anchored_text)
            plt.savefig(SAVE_PATH.format(exp,freq,su,roi))
            plt.savefig(SAVE_PATH.format(exp,freq,su,roi).replace('png','pdf'))


### Statistics by electrode or by patient
(not single-trial analysis) - Group analysis

In [ ]:
import numpy as np
from scipy.stats import wilcoxon
from os.path import join
from utils import subjects
import pandas as pd
from brainpipe.system import study

import statsmodels.api as sm
import statsmodels.formula.api as smf

###############################################################################
st = study('Olfacto')
rep_study = join(st.path, 'feature/TPSim_3groups_Enc/')
data_rep = join(rep_study, 'LinReg_stats_theta_v=1_elecs=all_hilbert=ok/')
csv_file = join(data_rep, 'All_subjects_ols_btw_theta_mem_groups_LinReg_width=None_step=None_ds.csv')
###############################################################################
conds = ['low','mid','high']
var_con = {'low':1,'mid':2,'high':3}
#rois = ['aHC_R','aHC_L','OFC_olf_R', 'OFC_olf_L', 'pPirT_L', 'pPirT_R']
rois = ['OFC_olf','aHC','pPirT','IFG']

df = pd.read_csv(csv_file)
df['labels'] = ['pPirT' if lab =='Amg' else lab for lab in df['labels']]
df['labels_RL'] = [x+'_R' if df['x'][i] > 0 else x+'_L' for i,x in enumerate(df['labels'])]
#rois = np.unique(df['labels'])

pvals, rois_sel = [], []
for roi in rois:
    df_roi = df.loc[df['labels']==roi]
    #df_roi2 = df_roi.loc[df_roi['channels'] != 'o2-o1'] #outlier
    #if (df_roi.groupby(['subjects']).mean().shape[0])>=4:
    rois_sel.append(roi)
    #print(df_roi[['subjects','channels','tps_low','tps_mid','tps_high']])

    df_take = df_roi
    #print(df_take)
    data = np.concatenate([df_take['tps_'+cond] for cond in conds])
    richness = np.concatenate([[var_con[cond]]*len(df_take['tps_'+cond]) for cond in conds])
    subjs = np.concatenate([df_take['subjects']]*len(conds))
    all_df = np.concatenate((subjs[:,np.newaxis],richness[:,np.newaxis],
                             data[:,np.newaxis]),axis=-1)
    df_stats = pd.DataFrame(all_df, columns=['subjs','richness','tps'])
    df_stats['richness'] = df_stats['richness'].astype('float')
    df_stats['tps'] = df_stats['tps'].astype('float')

    print(">>> processing ", roi)
    print(df_take[['tps_low','tps_mid','tps_high']].mean())
    #print(df_take[['tps_low','tps_mid','tps_high']].sem())

    md = smf.mixedlm("tps ~ richness", df_stats, groups = df_stats['subjs'])
    mdf = md.fit()
    pvals.append(mdf.pvalues[1])
    print(mdf.summary())

In [ ]:
            all_tps = np.concatenate(all_tps,axis=1)
            all_scores = np.concatenate(all_scores,axis=0)
            all_T, all_p = [], []
            for elec in range(nelecs):
                Y, X = np.array(all_tps[elec]), sm.add_constant(np.array(all_scores))
                model_ols = sm.OLS(Y,X).fit()
                #print(model_ols.summary())
                Tval, pval = np.round(model_ols.tvalues[1],2),np.round(model_ols.pvalues[1],3)
                all_T.append(Tval), all_p.append(pval)
            #Plot all values by subject
            tps_plot = np.mean(all_tps,axis=0)
            T, p = all_T[np.argmin(abs(np.array(all_T)))],np.max(np.array(all_p))
            fig, ax = plt.subplots(figsize=(3,3))
            #ax = sns.stripplot(x=all_scores, y=tps_plot, alpha=.25, zorder=1)
            ax = sns.violinplot(x=all_scores, y=tps_plot, alpha=.25, zorder=1, bw=0.5,scale='width')
            ax = sns.pointplot(x=all_scores, y =tps_plot, markers="d", scale=.75,
                              estimator=np.median, ci=None)